In [57]:
from ultralytics import YOLO
from sort import *
import cv2
import cvzone
import math

In [58]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [59]:
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("Videos/people.mp4")
mask = cv2.imread("Images/people_mask.png")    # Load an image that acts as a mask, limiting object detection to a specific area (e.g., where people are present).

In [60]:
#Tracking

tracker = Sort(max_age=20,min_hits=3 ,iou_threshold=0.3)

# Counting the objects that cross two specific areas: one for counting upwards and one for counting downwards.

limitsUp = [103, 161, 296, 161]  # Define the coordinates for the upper counting region (e.g., x1, y1, x2, y2) where objects moving upwards will be counted.
limitsDown = [600, 489, 735, 489]  # Define the coordinates for the lower counting region (e.g., x1, y1, x2, y2) where objects moving downwards will be counted.
totalCountUp = []  # List to store the IDs of objects that cross the upper counting region.
totalCountDown = []  # List to store the IDs of objects that cross the lower counting region.

In [ ]:
while True:
    success, img = cap.read()
    imageRegion = cv2.bitwise_and(img, mask)
    detections = np.empty((0,5))
    
    result = model(imageRegion, stream=True)
    for r in result:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w = x2-x1
            h = y2-y1

            conf = (math.ceil(box.conf[0] * 100))/100
            currentClass = classNames[int(box.cls[0])]

            if currentClass == "person" and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))


    resultTracker = tracker.update(detections)
    cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0,0,225),5)
    cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0,0,225),5)
    
    for results in resultTracker:
        x1, y1, x2, y2, Id = results
        x1, y1, x2, y2, Id = int(x1), int(y1), int(x2), int(y2), int(Id)

        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255,0,255))
        cvzone.putTextRect(img, f'{Id}' ,(max(0,x1), max(35,y1)), scale = 2, thickness = 3, offset=10)
        cx,cy = int(x1+w/2), int(y1+h/2)
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)


        #Up Count
        if limitsUp[0] < cx < limitsUp[2] and limitsUp[1]-10 < cy < limitsUp[3]+10:
            if Id not in totalCountUp:
                totalCountUp.append(Id)
                cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0,255,0),5)

        #Down Count
        if limitsDown[0] < cx < limitsDown[2] and limitsDown[1]-10 < cy < limitsDown[3]+10:
            if Id not in totalCountDown:
                totalCountDown.append(Id)
                cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0,255,0),5)

    cvzone.putTextRect(img, f'CountUp: {len(totalCountUp)} CountDown: {len(totalCountDown)}' ,(730,250), scale = 2, thickness = 3, offset=10)

    cv2.imshow("Video",img)
    cv2.waitKey(1)
        

    